In [9]:
import torchvision.models as models
resnet18 = models.resnet18().cuda()

import timm
mv2 = timm.create_model('mobilenetv2_100').cuda()

In [11]:
from torchsummary import summary

summary(resnet18, input_size=(3,244,244))
summary(mv2, input_size=(3,244,244))

C:\ProgramData\Miniconda3\envs\torch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 122, 122]           9,408
       BatchNorm2d-2         [-1, 64, 122, 122]             128
              ReLU-3         [-1, 64, 122, 122]               0
         MaxPool2d-4           [-1, 64, 61, 61]               0
            Conv2d-5           [-1, 64, 61, 61]          36,864
       BatchNorm2d-6           [-1, 64, 61, 61]             128
              ReLU-7           [-1, 64, 61, 61]               0
            Conv2d-8           [-1, 64, 61, 61]          36,864
       BatchNorm2d-9           [-1, 64, 61, 61]             128
             ReLU-10           [-1, 64, 61, 61]               0
       BasicBlock-11           [-1, 64, 61, 61]               0
           Conv2d-12           [-1, 64, 61, 61]          36,864
      BatchNorm2d-13           [-1, 64, 61, 61]             128
             ReLU-14           [-1, 64,

In [13]:
mv3 = timm.create_model('mobilenetv3_large_100').cuda()
summary(mv3, input_size=(3,244,244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 122, 122]             432
       BatchNorm2d-2         [-1, 16, 122, 122]              32
         Hardswish-3         [-1, 16, 122, 122]               0
            Conv2d-4         [-1, 16, 122, 122]             144
       BatchNorm2d-5         [-1, 16, 122, 122]              32
              ReLU-6         [-1, 16, 122, 122]               0
          Identity-7         [-1, 16, 122, 122]               0
            Conv2d-8         [-1, 16, 122, 122]             256
       BatchNorm2d-9         [-1, 16, 122, 122]              32
         Identity-10         [-1, 16, 122, 122]               0
DepthwiseSeparableConv-11         [-1, 16, 122, 122]               0
           Conv2d-12         [-1, 64, 122, 122]           1,024
      BatchNorm2d-13         [-1, 64, 122, 122]             128
             ReLU-14         [-1, 

In [14]:
mv2 = timm.create_model('mobilenetv2_100')
print(mv2)

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU6(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU6(inplace=True)
        (se): Identity()
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
    )
    (1): Sequential(
      (0): InvertedResidual(
        (conv_pw): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_

0it [00:00, ?it/s]

Extracting data\MNIST\raw\train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz


0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...


C:\ProgramData\Miniconda3\envs\torch\lib\site-packages\torchvision\datasets\mnist.py:335: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed).view(length, num_rows, num_cols)


Done!
[  1/100] train_loss: 0.82333 valid_loss: 0.29947
Validation loss decreased (inf --> 0.299474).  Saving model ...
[  2/100] train_loss: 0.36800 valid_loss: 0.21973
Validation loss decreased (0.299474 --> 0.219734).  Saving model ...
[  3/100] train_loss: 0.29981 valid_loss: 0.18481
Validation loss decreased (0.219734 --> 0.184809).  Saving model ...
[  4/100] train_loss: 0.25914 valid_loss: 0.16143
Validation loss decreased (0.184809 --> 0.161426).  Saving model ...
[  5/100] train_loss: 0.23563 valid_loss: 0.14466
Validation loss decreased (0.161426 --> 0.144659).  Saving model ...
[  6/100] train_loss: 0.21477 valid_loss: 0.13740
Validation loss decreased (0.144659 --> 0.137403).  Saving model ...
[  7/100] train_loss: 0.19750 valid_loss: 0.13469
Validation loss decreased (0.137403 --> 0.134688).  Saving model ...
[  8/100] train_loss: 0.18343 valid_loss: 0.12281
Validation loss decreased (0.134688 --> 0.122814).  Saving model ...
[  9/100] train_loss: 0.17432 valid_loss: 0.115

KeyboardInterrupt: 